# Vanilla RNNs, GRUs and the `scan` function

In this notebook, you will define the forward method for vanilla RNNs and GRUs. We will use the function `scan` to compute forward propagation for RNNs.

In [1]:
import numpy as np
from numpy import random
from time import perf_counter

from utils import sigmoid

# Part 1: Forward method for vanilla RNNs and GRUs

In this part, we'll  implement the forward propagation for a vanilla RNN and the same method for a GRU. we'll use a set of random weights and variables with the following dimensions:

- Embedding size (`emb`) : 128
- Hidden state size (`h_dim`) : 16

The weights `w_` and biases `b_` are initialized with dimensions (`h_dim`, `emb + h_dim`) and (`h_dim`, 1). We expect the hidden state `h_t` to be a column vector with size (`h_dim`,1) and the initial hidden state `h_0` is a vector of zeros.

In [2]:
# Random seed, 
random.seed(10)       
# Embedding size (size of the input vector x_t at each time step )          
emb = 128          
# Number of variables in the sequences (number of t's )
T = 256                        
# Hidden state dimension
h_dim = 16    
# Initial hidden state                  
h_0 = np.zeros((h_dim, 1))      
# Random initialization of weights and biases with the gaussian normal distribution
w1 = random.standard_normal((h_dim, emb+h_dim))
w2 = random.standard_normal((h_dim, emb+h_dim))
w3 = random.standard_normal((h_dim, emb+h_dim))
b1 = random.standard_normal((h_dim, 1))
b2 = random.standard_normal((h_dim, 1))
b3 = random.standard_normal((h_dim, 1))
X = random.standard_normal((T, emb, 1))
weights = [w1, w2, w3, b1, b2, b3]

## 1.1 Forward method for vanilla RNNs

The vanilla RNN cell is quite straight forward. Its most general structure is presented in the next figure: 

<img src="images/RNN.PNG" width="400"/>

Here are the computations made in a vanilla RNN cell :

In a vanilla RNN, the hidden state at time step $t$ ($h^{<t>}$) is computed as follows:

$$
h^{<t>} = g(W_{h}[h^{<t-1>}, x^{<t>}] + b_h)
\tag{1}
$$

Where:
- $h^{<t>}$ is the hidden state at time step $t$.
- $x^{<t>}$ is the input at time step $t$.
- $W_h$ is the weight matrix for the hidden state.
- $[h^{<t-1>}, x^{<t>}]$ represents the concatenation of the previous hidden state $h^{<t-1>}$ and the current input $x^{<t>}$ vertically.
- $b_h$ is the bias vector for the hidden state.
- $g$ is the activation function.

Additionally, the predicted output at time step $t$ ($\hat{y}^{<t>}$) is computed as follows:

$$
\hat{y}^{<t>} = g(W_{yh}h^{<t>} + b_y)
\tag{2}
$$

Where:
- $\hat{y}^{<t>}$ is the predicted output at time step $t$.
- $W_{yh}$ is the weight matrix for transforming the hidden state to the output.
- $b_y$ is the bias vector for the output.

The key operation here is the concatenation of $h^{<t-1>}$ and $x^{<t>}$ in equation (1), which combines the previous hidden state and the current input before applying the weight matrix $W_h$. In equation (2), the hidden state $h^{<t>}$ is used to compute the predicted output $\hat{y}^{<t>}$.


In [3]:
# Forward propagation for a a single vanilla RNN cell
def forward_vanilla_RNN(inputs, weights): 
    x, h_t = inputs

    # weights.
    wh, _, _, bh, _, _ = weights

    # new hidden state
    h_t = np.dot(wh, np.concatenate([h_t, x])) + bh
    h_t = sigmoid(h_t)

    return h_t, h_t

We omitted the computation of $\hat{y}^{<t>}$ for the sake of simplicity, to focus on the way that hidden states are updated here and in the GRU cell.

## 1.2 Forward method for GRUs

A GRU cell have more computations than the ones that vanilla RNNs have. You can see this visually in the following diagram:

<img src="images/GRU.PNG" width="400"/>

GRUs have relevance $\Gamma_r$ and update $\Gamma_u$ gates that control how the hidden state $h^{<t>}$ is updated on every time step. With these gates, GRUs are capable of keeping relevant information in the hidden state even for long sequences.(to avoid the vanishing gradient problem of the vanilla RNNs).

The equations needed for the forward method in GRUs are provided below: 

\begin{equation}
\Gamma_r=\sigma{(W_r[h^{<t-1>}, x^{<t>}]+b_r)}
\end{equation}

\begin{equation}
\Gamma_u=\sigma{(W_u[h^{<t-1>}, x^{<t>}]+b_u)}
\end{equation}

\begin{equation}
c^{<t>}=\tanh{(W_h[\Gamma_r*h^{<t-1>},x^{<t>}]+b_h)}
\end{equation}

\begin{equation}
h^{<t>}=\Gamma_u*c^{<t>}+(1-\Gamma_u)*h^{<t-1>}
\end{equation}

Now we will implement the forward method for a GRU cell by computing the update `u` and relevance `r` gates, and the candidate hidden state `c`. 

In [4]:
# Forward propagation for a single GRU cell
def forward_GRU(inputs, weights): 
    x, h_t = inputs

    # weights.
    wu, wr, wc, bu, br, bc = weights

    # Update gate
    u = np.dot(wu, np.concatenate([h_t, x])) + bu
    u = sigmoid(u)
    
    # Relevance gate
    r = np.dot(wr, np.concatenate([h_t, x])) + br
    r = sigmoid(r)
    
    # Candidate hidden state 
    c = np.dot(wc, np.concatenate([r * h_t, x])) + bc
    c = np.tanh(c)
    
    # New Hidden state h_t
    h_t = u* c + (1 - u)* h_t
    return h_t, h_t

Run the following cell to check your implementation.

# Part 2: Implementation of the `scan` function

The `scan` function is used for forward propagation in RNNs. It takes as inputs:

- `fn` : the function to be called recurrently (i.e. `forward_GRU`)
- `elems` : the list of inputs for each time step (`X`)
- `weights` : the parameters needed to compute `fn`
- `h_0` : the initial hidden state

`scan` goes through all the elements `x` in `elems`, calls the function `fn` with arguments ([`x`, `h_t`],`weights`), stores the computed hidden state `h_t` and appends the result to a list `y_hats`. 

In [5]:
# Forward propagation for RNNs
def scan(fn, elems, weights, h_0=None): 
    h_t = h_0
    y_hats = []
    for x in elems:
        y, h_t = fn([x, h_t], weights)
        y_hats.append(y)
    return y_hats, h_t

# Part 3: Comparison between vanilla RNNs and GRUs

GRUs perform more computations than vanilla RNNs, as they have 3 times more parameters. In the next two cells, we compute forward propagation for a sequence with 256 time steps (`T`) for an RNN and a GRU with the same hidden state `h_t` size (`h_dim`=16).  

In [6]:
# vanilla RNNs
tic = perf_counter()
y_hats, h_T = scan(forward_vanilla_RNN, X, weights, h_0)
toc = perf_counter()
vanilla_RNN_time=(toc-tic)*1000
print (f"It took {vanilla_RNN_time:.2f}ms to run the forward method for the vanilla RNN.")

It took 2.50ms to run the forward method for the vanilla RNN.


In [7]:
# GRUs
tic = perf_counter()
y_hats, h_T = scan(forward_GRU, X, weights, h_0)
toc = perf_counter()
GRU_time=(toc-tic)*1000
print (f"It took {GRU_time:.2f}ms to run the forward method for the GRU.")

It took 6.70ms to run the forward method for the GRU.


GRUs take more time to compute. This means that training and prediction would take more time for a GRU than for a vanilla RNN. However, GRUs allows us to propagate relevant information even for long sequences, so when selecting an architecture for NLP we should assess the tradeoff between computational time and performance. 